# Домашнее задание 6: классификация текстов

В этом домашнем задании вам предстоит построить классификатор текстов и поучаствовать в соревновании на Kaggle!
Первым делом вам предстоит построить хороший бейзлайн, а дальше пытаться улучшать, соревнуясь с другими участниками.


Ссылка на соревнование: https://www.kaggle.com/c/avito-category-prediction/overview
Данные для домашнего задания можно скачать на странице соревнования.
Чтобы ваше участие было засчитано, убедитесь, что имя в Leaderboard имеет вид: «Имя Фамилия номер_группы».

Оценивание:

Домашнее задание оценивается как обычно. Баллы указаны напротив заданий.

За соревнование даются бонусные баллы следующим образом.
Если вы пересекли baseline_2 на приватном лидерборде, ваша оценка равна 

10 - 10 * (i - 1) / M

где M — количество студентов, принявших участие в соревновании;

i — место (начиная с 1) студента в приватном лидерборде среди всех таких студентов.

Правила:

* Домашнее задание оценивается в 10 баллов.

* Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.

* Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.  

* В течение 3 суток после окончания соревнования в соответствующее задание на anytask необходимо прислать код, воспроизводящий ответы для посылки, фигурирующей в приватном лидерборде. В случае отсутствия кода, воспроизводящего результат, в установленный срок студенту выставляется 0 в качестве оценки за соревнование. Если не оговорено иное, использовать любые внешние данные в соревнованиях '''запрещено'''. Под внешними данными понимаются размеченные данные, где разметка имеет прямое отношение к решаемой задаче. Грубо говоря, сборник текстов с википедии не считается внешними данными.


Мы будем работать с датасетом объявлений Avito. Нам предстоит по заголовку и тексту объявления предсказывать категорию объявления.

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/home/seara/Desktop/Colab Notebooks/homework/hw06-text/train.csv')
test = pd.read_csv('/home/seara/Desktop/Colab Notebooks/homework/hw06-text/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/seara/Desktop/Colab Notebooks/homework/hw06-text/train.csv'

In [ ]:
train.head()

In [ ]:
train.shape

## Задание 1 (0.5 балла)

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их пустой строкой.

In [ ]:
print(train.isnull().any().any())
train.fillna("", inplace=True)
print(train.isnull().any().any())

In [ ]:
print(test.isnull().any().any())
test.fillna("", inplace=True)
print(test.isnull().any().any())

## Задание 2 (0.5 балла)
Сконкатенируйте заголовок и описание в единую строку и поместите результат в отдельный столбец. Таким образом мы будет работать с одним текстом, а не с двумя.

In [ ]:
test["title+description"] = test["title"] + " " + test["description"]
train["title+description"] = train["title"] + " " + train["description"]
test.drop(["title","description"], axis=1, inplace=True)
train.drop(["title","description"], axis=1, inplace=True)

In [ ]:
train.head()

## Задание 3 (0.5 балла)
Давайте немного посмотрим на наши данные. Визуализируйте (где явно просят) или выведете информацию о следующем:

1. Сколько всего уникальных классов необходимо предсказать?
2. Постройте столбчатую диаграмму распределения по классам. Нормализуйте график, чтобы видеть доли, а не абсолютные значения.
3. На сколько большой дисбаланс? Какой класс имеет самую большую долю в выборке (укажите класс и долю), какой класс наименьшую (укажите класс и долю).

In [ ]:
train["Category"].value_counts().size
# 50 уникальных категорий

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
sns.histplot(train, x="Category", linewidth=1,stat = "density", discrete=True)
plt.show()
# Дисбаланс серьёзный - три категориии (27,10,29) сильно доминируют

In [ ]:
train["Category"].value_counts(normalize=True)

In [ ]:
# Самая встречающаяся категория - 27(доля=0.180707)
# Самая невстречающаяся категория - 86(доля=0.000039)

## Задание 4 (0.5 балла)

Далее мы будем пока работать только с train частью. Для test части нам неизвестны истинные значения, и они понадобятся только тогда, когда мы будем отправлять решение на Kaggle.

1. Предобработайте данные (train часть) с помощью CountVectorizer.
2. Какого размера получилась матрица?


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer()
train_vec = cnt_vec.fit_transform(train["title+description"])
len(cnt_vec.get_feature_names())

In [ ]:
train_vec

## Задание 5 (0.5 балла)

В предыдущем пункте у вас должная была получится очень большая матрица. Это не дело.
Если вы взгляните на текст, то увидете, что там есть множество специальных символов.

Давайте также посмотрим на словарь, который получился в результате построения CountVectorizer, его можно найти в поле _vocabulary инстанса этого класса.

1. Найдите в этом словаре все слова, которые начинаются на цифру. Сколько таких слов нашлось?

2. Найдите все слова, которые начинаются на символы пунктуации. Сколько таких слов нашлось? 

In [ ]:
sum(x[0].isdigit() for x in list(cnt_vec.vocabulary_.keys())) # Слова, которые начинаются на цифру

In [ ]:
from string import punctuation
sum(x[0] in punctuation for x in list(cnt_vec.vocabulary_.keys())) # Слова, которые начинаются на символы пунктуации

## Задание 6 (2.5 балла)

В scikit-learn мы можем оценивать процесс подсчета матрицы через CountVectorizer. У CountVectorizer, как и у других наследников \_VectorizerMixin, есть аргумент tokenizer и preprocessor. preprocessor применится в самом начале к каждой строке вашего датасета, tokenizer же должен принять строку и вернуть токены.
Давайте напишем кастомный токенайзер, которые сделает все, что нам нужно: 

0. Приведет все буквы к нижнему регистру
1. Разобьет текст на токены с помощью word_tokenizer из пакета nltk
2. Удалит все токены содержащие числа и пунктуацию
3. Удалит все токены, которые перечислены в nltk.corpus.stopwords('russian')
4. Проведет стемминг с помощью SnowballStemmer

Продемонстрируйте работу вашей функии на самом первом описании товара в датасете.

Важно: так как функию будет необходимо запускать на большом корпусе, подумайте об эффективной реализации. Постарайтесь решить задачу в минимальное количество циклов и копирований.

Референсная реализация на компьютере автора работала 230 секунд на 100к строчек.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from string import punctuation
from string import digits

sw = set(stopwords.words('russian'))
stemmer = SnowballStemmer('russian')
noise = set(punctuation + digits)

def MyTokenizer(sentence):
    tokens = [stemmer.stem(x) for x in word_tokenize(sentence.lower()) if (len(set(x) & noise) == 0 and len(set([x]) & sw) == 0 and len(x)>2)]
    # Пересечение setов работает гораздо быстрее, чем "if x in noise", а результат тот же
    # Так же буду ещё отбрасывать короткие слова имеющие длину 2 символа и меньше
    return tokens

In [ ]:
MyTokenizer(train["title+description"][0])

In [ ]:
%%time
a = [MyTokenizer(train["title+description"][x]) for x in range(100000)]
# Примерно 120 секунд на 100к записей, с помощью пересечения сетов

## Задание 7 (1 балл)

1. Возьмите случайные 500к строк из датасета. Постройте по ним CountVectorizer с применением вашего токенизатора.
2. Разбейте полученную матрицу на train, test в отношении 4 к 1. Не забудьте про target переменную.
3. Обучите SGDClassifier на полученной выборке.
4. Посчитайте метрику accuracy

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
random_data = train.sample(500000)
cnt_vec = CountVectorizer(tokenizer=MyTokenizer, ngram_range=(1, 2))
X = cnt_vec.fit_transform(random_data["title+description"])

In [ ]:
# Буду добавлять во все векторизаторы параметр ngram_range=(1, 2), так(по моим экспериментам) они показывают наилучший результат
# Так же буду добавлять параметр n_jobs=-1 в классифайеры. Это позволяет им использовать все потоки процессора
# и обучаться в разы быстрее

In [ ]:
from sklearn.model_selection import train_test_split
y = random_data["Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337
)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

## Задание 8 (1 балл)

1. Повторите 7 задание, но с tf-idf векторизатором. Как изменилось качество.
2. Мы можем еще сильнее уменьшите размер нашей матрицы, если отбросим значения df близкие к единице. Скорее всего такие слова не несут много информации о категории, так как встречаются достаточно часто. Ограничьте максимальный df в параметрах TfIdfVectorizer, поставьте верхнюю границу равную 0.9. Как изменился размер матрицы, как изменилось качество?


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(tokenizer=MyTokenizer, ngram_range=(1, 2))
X = tfidf_vec.fit_transform(random_data["title+description"])
print(X)

<500000x3889340 sparse matrix of type '<class 'numpy.float64'>'
	with 25560887 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.model_selection import train_test_split
y = random_data["Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337
)

In [31]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.81745

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(tokenizer=MyTokenizer,max_df=0.9, ngram_range=(1, 2))
X = tfidf_vec.fit_transform(random_data["title+description"])
X

<500000x3889340 sparse matrix of type '<class 'numpy.float64'>'
	with 25560887 stored elements in Compressed Sparse Row format>

In [ ]:
# Ничего особо не изменилось, потому что токенайзер и так удаляет из выборки часто встречающиеся слова

In [34]:
from sklearn.model_selection import train_test_split
y = random_data["Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337
)

In [35]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.81728

In [ ]:
# Вообще, это конечно удивительно, что здесь TfidfVectorizer сработал хуже CountVectorizer.
# Поэксперементировав с векторайзерами я пришел к выводу, что в данной задаче TfidfVectorizer лучше.
# Максимальный результат на Kaggle с CountVectorizer 85-87%, когда с TfidfVectorizer мне удалось выбить 90%.
# (Эти результаты конечно же с чуть другим токенайзером и при обучении на всей выборке)

## Задание 9 (1 балл)
Еще один популяпный трюк, который позволит уменьшить количество признаков называется hashing trick. Его суть в том, то мы случайно группируем признаки ииии  ..... складываем их! А потом удаляем исходные признаки. В итоге все наши признаки это просто суммы исходных. Звучит странно, но это отлично работает. Давайте проверим этот трюк в нашем сеттинге.
Также при таком подходе вам не нужно хранить словарь token->index, что тоже иногда полезно.

1. Повторите задание 7 с HashingVectorizer, укажите количество фичей равное 30000.
2. Какой из трех подходов показал самый высокий результат?

In [45]:
from sklearn.feature_extraction.text import HashingVectorizer
hash_vec = HashingVectorizer(n_features=30000, ngram_range=(1, 2))
X = hash_vec.fit_transform(random_data["title+description"])
X

<500000x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 37591775 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.model_selection import train_test_split
y = random_data["Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337
)

In [47]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.80254

In [ ]:
# HashingVectorizer не смог проявить себя достойно.
# В этом ноутбуке CountVectorizer показал самый высокий результат.
# Но как я уже написал выше, это иллюзия))

## Задание 10 (1.5 балла)

Пришло время выйти в мир. Отправляйтесь на Kaggle Inclass и сделайте первую посылку. Там вы найдете инструкцию как сформировать файл с предсказаниями и отправить его.

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
random_data = train.sample(500000)
cnt_vec = CountVectorizer(tokenizer=MyTokenizer)
X = cnt_vec.fit_transform(random_data["title+description"])

In [50]:
from sklearn.model_selection import train_test_split
y = random_data["Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337
)

In [51]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

0.85699

In [52]:
test_data = cnt_vec.transform(test["title+description"])
pred = clf.predict(test_data)
data = {'Id':test["itemid"].to_list(),
        'Category':pred}
df = pd.DataFrame(data)
df.to_csv("lol.csv",index=False)

In [ ]:
# Тут я написал код своей первой попытки загрузки на kaggle. Результат - 0.85589